In [3]:
import numpy as np
import pandas as pd
import math

In [111]:
class Binomial_Trees():
    def __init__(self, stock_price, volatility, time_steps, time2expiry, interest_free_rate, strike_price,option_type = 'ec'):
        self.S0 = stock_price
        self.K = strike_price
        self.sigma = volatility
        self.ts = min(time_steps,10)
        self.T = time2expiry
        self.time_steps = time_steps
        self.delta_t = self.T/time_steps
        self.r = interest_free_rate
        self.u = math.exp(volatility*math.sqrt(self.delta_t))
        self.d = math.exp(-volatility*math.sqrt(self.delta_t))
        self.p = (math.exp(self.r*self.delta_t) - self.d)/(self.u - self.d)
        self.spt = []
        self.option_type = option_type
        self.stock_prices()
        self.opt = []
        self.option_prices()

    def stock_prices(self):
        self.spt.append([self.S0])
        for ts in range(1,self.time_steps):
            self.spt.append(self.stock_price_at_timestept(ts))
            
    def stock_price_at_timestept(self, timestep):
        stock_prices = []
        for s in self.spt[timestep-1]:
            for move in range(2):
                if(move==0):
                    stock_prices.append(s*self.u)
                else:
                    stock_prices.append(s*self.d)
        return stock_prices

    def print_sp(self):
        num_values = 2**(self.time_steps-1)
        for spt in reversed(self.spt):
            for sp in spt:
                print(sp, end="   ")
            print()

    def print_op(self):
        num_values = 2**(self.time_steps-1)
        for opt in self.opt:
            for sp in opt:
                print(sp, end="   ")
            print()
            
    def option_value_mature(self):
        option_value = []
        assert self.option_type in ['ec','ep','ac','ap']
        if(self.option_type=='ec'):
            for sp in self.spt[-1]:
                option_value.append(sp-self.K if sp-self.K>0 else 0.0)
        elif(self.option_type=='ep'):
            for sp in self.spt[-1]:
                option_value.append(self.K-sp if sp-self.K<0 else 0.0)
        return option_value

    def option_prices(self):
        self.opt.append(self.option_value_mature())
        for ts in range(1,self.time_steps):
            self.opt.append(self.option_value_at_timestept(ts))

    def option_value_at_timestept(self, timestep):
        option_value = []
        s = self.opt[timestep-1]
        for idx in range(0,len(self.opt[timestep-1])//2):
            fu = s[2*idx]
            fd = s[2*idx + 1]
            f = math.exp(-self.r*self.delta_t)*(self.p*fu + (1-self.p)*fd)
            if self.option_type=='ec' or self.option_type=='ep':
                option_value.append(f)
        return option_value

In [112]:
bt = Binomial_Trees(20, 1, 3, 3, 0.12, 21, option_type='ec')

In [113]:
bt.print_sp()

24.200000000000003   19.8   19.8   16.2   
22.0   18.0   
20   


In [114]:
bt.print_op()

3.200000000000003   0.0   0.0   0.0   
3.2283457112729343   0.0   
3.256942509841981   
